In [2]:
import pandas as pd

This report is responsible for [Mondo HPO Google Spreadsheet generation](https://github.com/monarch-initiative/mondo/issues/7359) 

In [9]:
!cd .. && ./run.sh make ./tmp/mondo-hpo-merged-lexmatch.sssom.tsv IMP=false MIR=false

latest: Pulling from obolibrary/odkfull
Digest: sha256:40dd8d5717db953868a6186222831612847a20a3d935d88b8cbb7ba1718f849a
Status: Image is up to date for obolibrary/odkfull:latest
docker.io/obolibrary/odkfull:latest

What's Next?
  View summary of image vulnerabilities and recommendations → docker scout quickview obolibrary/odkfull:latest
Running ODK with 8 GB of memory, using odkfull:latest.
Makefile:682: warning: overriding recipe for target 'mirror-hgnc'
Makefile:571: warning: ignoring old recipe for target 'mirror-hgnc'
mondo.Makefile:1019: warning: overriding recipe for target 'help'
Makefile:1394: warning: ignoring old recipe for target 'help'
make: 'tmp/mondo-hpo-merged-lexmatch.sssom.tsv' is up to date.


In [3]:
mondo_hpo_sssom_file="../tmp/mondo-hpo-merged-lexmatch.sssom.tsv"
mondo_hpo_sssom = pd.read_csv(mondo_hpo_sssom_file, sep='\t', encoding='utf-8', comment='#')

In [4]:
mondo_hpo_sssom.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,BFO:0000016,disposition,skos:closeMatch,PATO:0001727,disposition,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,disposition
1,BFO:0000019,quality,skos:closeMatch,PATO:0000001,quality,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,quality
2,BFO:0000023,role,skos:closeMatch,CHEBI:50906,role,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,role
3,CHEBI:16134,ammonia,skos:closeMatch,NCBITaxon:29189,Ammonia,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,ammonia
4,CHEBI:24870,ion,skos:closeMatch,MONDO:0044688,isolated optic neuritis,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,ion


The next piece of code was written in two steps because I wanted to ensure the MONDO ID and HPO ID in both conditions (when subject_id starts with HP or object_id starts with HP and the opposite way for MONDO IDs)

In [5]:
hpo_mondo_sssom_filtered = mondo_hpo_sssom.query('(subject_id.str.startswith("HP:").values & object_id.str.startswith("MONDO:").values)')
hpo_mondo_sssom_selector = {'object_id': 'mondo_id', 'object_label': 'mondo_label', 'subject_id': 'hpo_id', 'subject_label': 'hpo_label', 'mapping_justification':'justification'}
hpo_mondo_sssom_filtered_renamed = hpo_mondo_sssom_filtered.rename(columns=hpo_mondo_sssom_selector)[[*hpo_mondo_sssom_selector.values()]]

mondo_hpo_sssom_filtered = mondo_hpo_sssom.query('(subject_id.str.startswith("MONDO:").values & object_id.str.startswith("HP:").values)')
mondo_hpo_sssom_selector = {'subject_id': 'mondo_id', 'subject_label': 'mondo_label', 'object_id': 'hpo_id', 'object_label': 'hpo_label', 'mapping_justification':'justification'}
mondo_hpo_sssom_filtered_renamed = mondo_hpo_sssom_filtered.rename(columns=mondo_hpo_sssom_selector)[[*hpo_mondo_sssom_selector.values()]]

mondo_hpo_sssom_filtered_combined = pd.concat([hpo_mondo_sssom_filtered_renamed, mondo_hpo_sssom_filtered_renamed])


mondo_hpo_sssom_filtered_combined.to_csv('./mondo_hpo.tsv', sep='\t', index=None, encoding='utf-8')